# Motivation
论文原文：[《Deep Learning over Multi-field Categorical Data: A Case Study on User Response Prediction》](https://arxiv.org/pdf/1601.02376)

参考学习地址：[推荐系统系列（三）：FNN理论与实践](https://zhuanlan.zhihu.com/p/89850096)

FM通常指对特征进行二阶交叉，FNN（Factorization Machine supported Neural Network）参照CNN通过相邻层连接扩大感受野的做法，使用DNN来对FM显示表达的二阶交叉特征进行再交叉，进而产生更高阶的特征组合。思想就是在FM上接入若干全连接层，利用DNN对特征进行隐式交叉。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/fnn.png" width = "600" height = "400" alt="FNN" align=center />

## 两阶段训练
为了加速模型收敛，FNN两阶段训练，将FM作为先验知识加入到模型中，防止因为数据稀疏带来的歧义造成模型参数偏差。
* Step1: 针对人物构建FM模型，完成模型参数学习
* Step2: 将FM参数作为FNN底层参数的初始值。

## 分域
结构图中可以看到，特征进行输入之前首先进行了分域操作，这种方式也成了后续处理高维稀疏性数据的通用做法，目的是为了减少模型参数量，与FM计算过程保持一致。模型中 DenseRealLayer 将FM产出的低位稠密特征向量进行简单拼接，作为下一全连接层输入，蚕蛹tanh激活函数，最终使用sigmoid将输出压缩至0-1作为预测。

## 一些思考
* FM的特征组合使用的是隐向量点积，而NN本质将输入向量的所有元素进行加权求和，也不会区分特征Field。也就是说 <font color="red">FNN中高阶特征组合使用的是全部隐向量元素相加的方式（分域的意义仅体现在了先验注入的部分，最后的概率输出会受NN内部影响）</font>。说到底，在历届特征组合的层面上FNN与FM存在Gap，这也是后续PNN改进的动力。
* FNN的底层参数通过FM预训练获取，反向传播更新参数的学习率不宜过大、或进行clip、或layer-wise学习率（底层低，上层高），否则FM先验信息很容易被抹去